# Environment Setting

In [0]:
! pip3 install --upgrade tensorflow-gpu
! pip3 install --upgrade keras

### Test if tf can see GPU

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(f'Fonnd GPU device at {device_name}')

Fonnd GPU device at /device:GPU:0


# Datasets Preparation

In [0]:
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout

# import warnings
# warnings.filterwarnings('ignore')

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/MRG Project/data'

train_path = f'{path}/train'
valid_path = f'{path}/validation'
test_path = f'{path}/test'

In [0]:
# train: valid: test = 6: 2: 2
# train = 600, valid = test = 200

img_size = (224,224)
input_shape =(224,224,3)
num_classes = 10


train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=img_size, 
                                                         class_mode='sparse', batch_size=50)

valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=img_size, 
                                                         class_mode='sparse', batch_size=20)

test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=img_size, 
                                                        class_mode='sparse', batch_size=10)

Found 580 images belonging to 10 classes.
Found 196 images belonging to 10 classes.
Found 195 images belonging to 10 classes.


# Train & Predict

## Model1 (Jump to Model2/VGG...)
Based on: https://www.youtube.com/watch?v=LhEMXbjGV_4

In [0]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

In [0]:
model.fit_generator(train_batches, steps_per_epoch=12, 
                    validation_data=valid_batches, validation_steps=10, epochs=20)    # steps_per_epoch = 600/50 = 12

In [0]:
model.evaluate_generator(test_batches, steps=20)

[8.664322853088379, 0.28205129504203796]

## Model 2
Based on: https://medium.com/x8-the-ai-community/audio-classification-using-cnn-coding-example-f9cbd272269e

In [0]:
model = Sequential()

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

In [0]:
model.fit_generator(train_batches, steps_per_epoch=12, 
                    validation_data=valid_batches, validation_steps=10, epochs=10)

## Model3 -- VGG16
Based on: https://www.youtube.com/watch?v=oDHpqu52soI

### 1.Adam(lr=.0001)
epoch: 70

train: 0.76

valid: 0.54

test: 0.56

In [0]:
from keras.applications.vgg16 import VGG16

vgg16_model = VGG16()
vgg16_model.layers.pop()

from keras.layers import InputLayer

new_model = Sequential()
new_model.add(InputLayer(input_shape=input_shape))

for layer in vgg16_model.layers:
  new_model.add(layer)

for layer in new_model.layers:
  layer.trainable=False

new_model.add(Dense(num_classes, activation='softmax'))

new_model.summary()

from keras.optimizers import Adam

new_model.compile(optimizer=Adam(lr=.0001),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [0]:
new_model.fit_generator(train_batches, steps_per_epoch=12, 
                    validation_data=valid_batches, validation_steps=10, epochs=20)

In [0]:
new_model.evaluate_generator(test_batches, steps=20)

### 2.'adam' 
epoch: 90

train_acc = 1

valid_acc = 0.6276

test_acc = 0.63

In [0]:
from keras.applications.vgg16 import VGG16

vgg16_model = VGG16()
vgg16_model.layers.pop()

from keras.layers import InputLayer

new_model = Sequential()
new_model.add(InputLayer(input_shape=input_shape))

for layer in vgg16_model.layers:
  new_model.add(layer)

for layer in new_model.layers:
  layer.trainable=False

new_model.add(Dense(num_classes, activation='softmax'))

new_model.summary()

new_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [0]:
new_model.fit_generator(train_batches, steps_per_epoch=12, 
                    validation_data=valid_batches, validation_steps=10, epochs=10)

In [0]:
new_model.evaluate_generator(test_batches, steps=20)

In [0]:
# # How to save a model

# new_model.save('/content/drive/My Drive/Colab Notebooks/MRG Project/models/m1.h5')

# from keras.models import load_model

# model = load_model('/content/drive/My Drive/Colab Notebooks/MRG Project/models/m1.h5')

# model.evaluate_generator(test_batches, steps=20)

### 3.SGD (as vgg16)
Bad results!

In [0]:
from keras.applications.vgg16 import VGG16

vgg16_model = VGG16()
vgg16_model.layers.pop()

from keras.layers import InputLayer

new_model = Sequential()
new_model.add(InputLayer(input_shape=input_shape))

for layer in vgg16_model.layers:
  new_model.add(layer)

for layer in new_model.layers:
  layer.trainable=False

new_model.add(Dense(num_classes, activation='softmax'))

new_model.summary()

from keras.optimizers import SGD

new_model.compile(optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [0]:
new_model.fit_generator(train_batches, steps_per_epoch=12, 
                    validation_data=valid_batches, validation_steps=10, epochs=50)

Epoch 1/50
12/12 [==============================] - 8s 706ms/step - loss: 454.0563 - accuracy: 0.1167 - val_loss: 521.3250 - val_accuracy: 0.1327
Epoch 2/50
12/12 [==============================] - 6s 472ms/step - loss: 389.4902 - accuracy: 0.2417 - val_loss: 120.4024 - val_accuracy: 0.2500
Epoch 3/50
12/12 [==============================] - 7s 590ms/step - loss: 117.5171 - accuracy: 0.3625 - val_loss: 89.2627 - val_accuracy: 0.4082
Epoch 4/50
12/12 [==============================] - 6s 532ms/step - loss: 84.3398 - accuracy: 0.4292 - val_loss: 64.2331 - val_accuracy: 0.3827
Epoch 5/50
12/12 [==============================] - 7s 544ms/step - loss: 74.4850 - accuracy: 0.4583 - val_loss: 108.2851 - val_accuracy: 0.4949
Epoch 6/50
12/12 [==============================] - 6s 499ms/step - loss: 66.7729 - accuracy: 0.4792 - val_loss: 139.8829 - val_accuracy: 0.3776
Epoch 7/50
12/12 [==============================] - 6s 522ms/step - loss: 76.6596 - accuracy: 0.5333 - val_loss: 53.1737 - val_ac